## SAS Tip Tuesday: Submit SAS Code with SASPy
*03/11/2025*

- *Stu Sztukowski*
- *https://linkedin.com/in/StatsGuy* | *https://github.com/stu-code*

***

One great thing about SASPy is that it also lets you run arbitrary SAS code. It's as easy as starting a connection and running sas.submit(). You can get output in list or HTML - your choice! With IPython.display you can easily display the returned ODS output if needed.

When you're working with SASPy, you're in a full SAS session. This means you can do anything SAS can do:
- Run SQL queries
- Run the DATA Step
- Run PROCs
- Create libraries
- Connect to CAS
- Run CAS Actions with PROC CAS
- etc.

In [1]:
import saspy
from IPython.display import HTML

## Start a SAS session

You can configure the connection types you want within ~./config/saspy/sascfg_personal.py. 

We'll start a local connection to SAS installed on a local Windows machine, but you can easily connect to SAS Viya, SAS Grid, SAS Mainframe, or a remote SAS server. 

For more information on how to change how you connect to SAS, see:
https://sassoftware.github.io/saspy/getting-started.html#start-a-sas-session

In [ ]:
sas = saspy.SASsession(cfgname='winlocal')

## Try a DATA Step

Now that we have started a SAS session, we can run SAS any way we'd like. Let's run some SAS code directly to import data from a URL and print it.

In [3]:
res = sas.submit(code=
'''
filename in url "https://github.com/stu-code/sas-tips/raw/refs/heads/main/data/insurance.csv";

proc import
    file=in
    out=insurance
    dbms=csv
    replace;
run;
'''
)

# Checking the log

Now we've imported our data as work.insurance. To confirm this, you can easily check the log by printing sas.lastlog()

In [ ]:
print(sas.lastlog())

# Displaying Output

Let's print the first 10 observations using PROC PRINT. We'll specify that we want HTML output and print it so it looks nice. Since it comes back as HTML, we'll need to use the HTML function to see it from IPython.display.

To get the results, use `res['LST']`

In [ ]:
res = sas.submit(code=
'''
proc print data=insurance(obs=10);
run;
'''
)

HTML(res['LST'])

In [ ]:
res = sas.submit(
'''
proc glmselect data=insurance plots=all;
    class sex smoker region;
    model charges = age sex bmi children smoker region 
        / selection=stepwise(select=sl choose=adjrsq sle=0.1 sls=0.05) 
          showpvalues;
run;
'''
)

HTML(res['LST'])

# Data Manipulation: DATA Step and SQL

With sas.submit, you can do data manipulation with the DATA Step and SQL - whichever you like, in any order you like. You can mix and match Python and SAS. There is no limit to what order you do things and in what language you decide to do them.

In [7]:
res = sas.submit(code=
'''
data insurance2;
    set insurance;
    sex     = upcase(sex);
    region  = upcase(region);
    charges = round(charges);
run;
'''
)

In [ ]:
insurance2 = sas.sasdata('insurance2')
insurance2.head()

In [9]:
res = sas.submit(code=
'''
proc sql;
    create table us_counties as
        select t1.id as county_id
             , t2.idname as county 
             , t1.county as county_id
             , t1.statecode
             , t1.x
             , t1.y
        from mapsgfk.us_counties as t1
        LEFT JOIN
             mapsgfk.us_counties_attr as t2
        ON t1.id = t2.id
    ;
quit;
'''              
)

In [ ]:
us_counites = sas.sasdata('us_counties')
us_counites.head()